In [ ]:
!pip install torch==2.7.0 torchvision==0.22.0 torchaudio==2.7.0 --index-url https://download.pytorch.org/whl/cu126

!pip install -q transformers torchinfo datasets fsspec huggingface_hub evaluate xformers

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 93.3 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.6.0+cu124
    Uninstalling torchaudio-2.6.0+cu124:
      Successfully uninstalled torchaudio-2.6.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os, torch, random
from transformers import (
    AutoModel, AutoTokenizer,
    AutoModelForCausalLM, BitsAndBytesConfig
)
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_dataset
import pathlib, torch, datetime as dt
import xformers
from datetime import datetime

In [9]:
CKPT_DIR = pathlib.Path("/content/drive/MyDrive/fMRIwork/checkpoints")
CKPT_DIR.mkdir(exist_ok=True, parents=True)

In [10]:
path = "/content/drive/MyDrive/fMRIwork/Projector_Training_Data/c4training_small.txt"
with open(path, encoding="utf-8") as f:
    corpus = [line.strip() for line in f]
print(f"Loaded {len(corpus):,} lines")


Loaded 50,000 lines


In [11]:
corpus[:5]

['Beginners BBQ Class Taking Place in Missoula!',
 "Discussion in 'Mac OS X Lion (10.7)' started by axboi87, Jan 20, 2012.",
 'Foil plaid lycra and spandex shortall with metallic slinky insets. Attached metallic elastic belt with O-ring. Headband included. Great hip hop or jazz dance costume. Made in the USA.',
 'How many backlinks per day for new site?',
 'The Denver Board of Education opened the 2017-18 school year with an update on projects that include new construction, upgrades, heat mitigation and quality learning environments.']

In [12]:
# ------------------------------ CONFIG --------------------------------
NEOBERT_CKPT   = "chandar-lab/NeoBERT" #"bert-base-uncased"
LLAMA_CKPT     = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
BATCH_SIZE     = 128 #64
LR             = 2e-4
NUM_EPOCHS     = 10
MAX_LEN        = 64                             # truncate long sentences
DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"
SEED           = 42

torch.manual_seed(SEED); random.seed(SEED)

In [13]:
# -------------------------- Load models -----------------------------
print("Loading NeoBERT …")
neo_tok  = AutoTokenizer.from_pretrained(NEOBERT_CKPT)
neo_bert = AutoModel.from_pretrained(NEOBERT_CKPT,trust_remote_code=True).requires_grad_(False).to(DEVICE)

print("Loading TinyLlama …")
bnb_cfg  = BitsAndBytesConfig(load_in_8bit=True)
llm_tok  = AutoTokenizer.from_pretrained(LLAMA_CKPT)
llm = AutoModelForCausalLM.from_pretrained(
    LLAMA_CKPT,
    device_map="auto",
    torch_dtype=torch.float32,      # load weights in half-precision
).eval().requires_grad_(False)


bert_dim  = neo_bert.config.hidden_size                   # 768
llama_dim = llm.config.hidden_size                        # 2048


Loading NeoBERT …


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading TinyLlama …


In [15]:
# ------------------------ 2. Projector net -----------------------------
class Bert2Llama(nn.Module):
    """4-layer GELU MLP: 768 ➜ 2048"""
    def __init__(self, in_d=bert_dim, out_d=llama_dim, h_d=1536, n=5): #n=4):
        super().__init__()
        layers = []
        for i in range(n):
            layers.append(nn.Linear(in_d if i == 0 else h_d,
                                    out_d if i == n-1 else h_d))
            if i < n-1:
                layers.append(nn.GELU())
        self.net = nn.Sequential(*layers)

    def forward(self, x):          # x: (B, T, 768)
        return self.net(x)         #    (B, T, 2048)

projector = Bert2Llama().to(DEVICE)

In [16]:
def collate(batch):
    # Encode with both tokenizers
    neo_enc  = neo_tok(batch, padding=True, truncation=True,
                       max_length=MAX_LEN, return_tensors="pt")
    llama_enc= llm_tok(batch, padding=True, truncation=True,
                       max_length=MAX_LEN, return_tensors="pt")
    return neo_enc, llama_enc

loader = DataLoader(corpus, batch_size=BATCH_SIZE,
                    shuffle=True, collate_fn=collate, num_workers=2)

In [17]:
# ---------------------- Training loop -------------------------------
optim = torch.optim.AdamW(projector.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss(ignore_index=llm_tok.pad_token_id)
global_step  = 0

print("\n===== Training projector =====")
for epoch in range(1, NUM_EPOCHS + 1):
    total, n_tok = 0.0, 0
    for neo_enc, llama_enc in loader:

        # Move to GPU
        neo_enc  = {k:v.to(DEVICE) for k,v in neo_enc.items()}
        llama_ids= llama_enc["input_ids"].to(DEVICE)

        with torch.no_grad():
            neo_out = neo_bert(**neo_enc).last_hidden_state  # (B,T,768)

        # Align sequence lengths
        T = min(neo_out.size(1), llama_ids.size(1))
        neo_out   = neo_out[:, :T, :]                        # (B,T,768)
        labels    = llama_ids[:, :T]                         # (B,T)

        # Forward through projector & LLaMA
        proj_emb = projector(neo_out)                  # (B, T, 2048)
        proj_emb = proj_emb.to(dtype=next(llm.parameters()).dtype)
        out = llm(inputs_embeds=proj_emb, labels=labels)

        loss = out.loss
        loss.backward(); optim.step(); optim.zero_grad()

        global_step += 1
        total  += loss.item() * labels.numel()
        n_tok  += labels.numel()

        if global_step % 1_000 == 0:
            torch.save({
                "epoch":      epoch,
                "step":       global_step,
                "proj_state": projector.state_dict(),
                "opt_state":  optim.state_dict(),
                "rng_state":  torch.get_rng_state(),
                "rng_state_cuda": torch.cuda.get_rng_state_all()
                                   if torch.cuda.is_available() else None,
            }, CKPT_DIR / f"step_{global_step:07d}.pt")

    # Epoch checkpoint
    torch.save(
        {
            "epoch": epoch,
            "proj_state": projector.state_dict(),
            "opt_state":  optim.state_dict(),
            "rng_state":  torch.get_rng_state(),
        },
        CKPT_DIR / f"epoch_{epoch:02d}.pt"
    )

    ppl = torch.exp(torch.tensor(total / n_tok))
    print(f"Epoch {epoch}: loss {total/n_tok:.4f}  |  ppl {ppl:.2f}")


=== Training projector ===
Epoch 1: loss 4.6100  |  ppl 100.48
Epoch 2: loss 3.2884  |  ppl 26.80
Epoch 3: loss 2.9417  |  ppl 18.95
Epoch 4: loss 2.7392  |  ppl 15.47
Epoch 5: loss 2.5882  |  ppl 13.31
Epoch 6: loss 2.4486  |  ppl 11.57
Epoch 7: loss 2.3302  |  ppl 10.28
Epoch 8: loss 2.2125  |  ppl 9.14
Epoch 9: loss 2.1238  |  ppl 8.36
Epoch 10: loss 2.0250  |  ppl 7.58


In [18]:
def neo2llama(text, max_new=0):
    """Feed NeoBERT → projector → LLaMA.
       If max_new==0 → teacher forcing (should echo).
       Else           → free generation (should still echo first part)."""
    with torch.no_grad():
        neo_in = neo_tok(text, return_tensors="pt").to(DEVICE)
        bert_h = neo_bert(**neo_in).last_hidden_state       # (1, B_len, 768)
        bert_h = bert_h.to(dtype=projector.net[0].weight.dtype)
        proj_h = projector(bert_h)

        llama_in = llm_tok(text, return_tensors="pt").to(DEVICE)
        labels   = llama_in["input_ids"]  # (1, L_len)


        mask   = torch.ones(proj_h.size()[:2], dtype=torch.long, device=DEVICE)


        if max_new == 0:
            T = min(proj_h.size(1), labels.size(1))
            proj_h = proj_h[:, :T, :]
            labels = labels[:, :T]


            out    = llm(inputs_embeds=proj_h, labels=labels)
            ids    = out.logits.argmax(-1)


        else:
            ids = llm.generate(
                inputs_embeds=proj_h,
                input_ids=torch.full(
                  (1, proj_h.size(1)), fill_value=llm_tok.bos_token_id, device=DEVICE
                ),
                attention_mask=mask,
                max_new_tokens=max_new,
                eos_token_id=llm_tok.eos_token_id,
                pad_token_id=llm_tok.pad_token_id,
                do_sample=False,
            )

    return llm_tok.decode(ids[0], skip_special_tokens=True)



In [19]:
test_sentences = [
    "Quantum computers promise speed-ups that classical machines cannot match.",
    "The Aurora Borealis shimmered like green silk across the night sky.",
    "After twelve grueling innings, the underdogs finally clinched the pennant.",
    "Please email the revised PDF to marketing by 09:30 a.m. tomorrow.",
    "Serendipity often favors the curious, the prepared, and the persistent.",
    "“We’re out of espresso beans,” whispered the barista, glancing at the queue.",
    "A tiny gecko clung effortlessly to the laboratory’s glass wall.",
    "GDP growth slowed to 2.1 percent in Q2, missing analysts’ expectations.",
    "Einstein’s handwriting was notoriously hard to read, even for his assistants.",
    "The ancient oak, felled by lightning, revealed concentric rings of drought.",
    "Machine-learning models excel at interpolation but struggle with extrapolation.",
    "During the blackout, neighbors gathered in the hallway and shared candles.",
    "My passport expires in eleven months; I’d better renew it soon.",
    "Some mushrooms glow faintly in the dark due to bioluminescent enzymes.",
    "Arctic sea-ice extent hit a record low, alarming climate scientists worldwide.",
    "He typed rm -rf /*, realized the mistake, and pulled the network cable.",
    "To brew oolong correctly, let the kettle cool to eighty-five degrees Celsius.",
    "The violin’s G string snapped mid-concerto, yet the soloist played on.",
    "Cicadas emerge every seventeen years, blanketing trees in vibrating sound.",
    "Static friction exceeds kinetic friction—hence the sudden jerk when boxes slide.",
    "In 2029 NASA aims to redirect a small asteroid, proving planetary-defense tech.",
    "Her résumé boasted fluency in Sinhala, Japanese, and American Sign Language.",
    "The bookstore’s resident cat sleeps on whatever stack you’re trying to reach.",
    "Deep-sea vents host microbes that survive without sunlight, using chemosynthesis.",
    "“Ctrl + Z is my best friend,” the developer joked after a bad deploy.",
    "Sourdough starters are tiny ecosystems of yeast, lactobacilli, and hype.",
    "The committee postponed the vote until October 3rd, citing budget ambiguities.",
    "Long-form journalism thrives when readers trust slow, meticulous storytelling.",
    "A single photon triggered the avalanche photodiode, logging a digital ‘1’.",
    "The chef garnished the ramen with nori, scallions, and a seven-minute egg.",
    "For centuries, cartographers added mythical islands to fill white space.",
    "The smartwatch recorded 10,438 steps, but she still felt sedentary.",
    "PyTorch 2.3 introduced compile(), merging graph-mode speed with eager simplicity.",
    "Call me old-fashioned, but I still sync MP3s instead of streaming music.",
    "Two koalas dozed in the eucalyptus, oblivious to the drone overhead.",
    "His haiku read: autumn wind / carries lost umbrellas / into gray rivers.",
    "The conference Wi-Fi password was taped, in Comic Sans, to every chair.",
    "Riemann’s hypothesis remains unproven, yet primes obey its silent rhythm.",
    "“No spoilers!” she yelled, scrolling past the season-finale tweets.",
    "Solar panels blanketed the barn roof, outshining the rusty weather vane.",
    "A cashierless store feels eerie until you realize how fast you can leave."
]


In [22]:
#-------------------------- Generation Test ------------------------------
for sentence in test_sentences:
  print("Input sentence (Ground Truth):   ", sentence)
  print(f"'Generated:  ', {neo2llama(sentence, max_new=0)}\n")

Input sentence (Ground Truth):    Quantum computers promise speed-ups that classical machines cannot match.
'Generated:  ', Theon Comput promise speed of ups that that machines cannot..

Input sentence (Ground Truth):    The Aurora Borealis shimmered like green silk across the night sky.
'Generated:  ', The skyal ofalis themermerled like likekkk night night sky

Input sentence (Ground Truth):    After twelve grueling innings, the underdogs finally clinched the pennant.
'Generated:  ', The areteenteenlingingsies, the underubs finally finally finally the the.

Input sentence (Ground Truth):    Please email the revised PDF to marketing by 09:30 a.m. tomorrow.
'Generated:  ', The submit your newed PDF PDFetingeting 90::0....

Input sentence (Ground Truth):    Serendipity often favors the curious, the prepared, and the persistent.
'Generated:  ', The areitions,. leadss.. the curious. and,.ive

Input sentence (Ground Truth):    “We’re out of espresso beans,” whispered the barista, glancing a

In [23]:
save_dir = "/content/drive/MyDrive/fMRIwork"
os.makedirs(save_dir, exist_ok=True)

# Timestamp versioning
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"generation_test_{timestamp}.txt"
filepath = os.path.join(save_dir, filename)

# Save generation test and config
with open(filepath, 'w') as f:
    f.write("# ------------------ Model Configuration ------------------\n")
    f.write(f"NEOBERT_CKPT  = {NEOBERT_CKPT}\n")
    f.write(f"LLAMA_CKPT    = {LLAMA_CKPT}\n")
    f.write(f"BATCH_SIZE    = {BATCH_SIZE}\n")
    f.write(f"LR            = {LR}\n")
    f.write(f"NUM_EPOCHS    = {NUM_EPOCHS}\n")
    f.write(f"MAX_LEN       = {MAX_LEN}\n")
    f.write(f"DEVICE        = {DEVICE}\n")
    f.write(f"SEED          = {SEED}\n")
    f.write("\n# ------------------ Generation Test Output ------------------\n")

    for sentence in test_sentences:
        output = neo2llama(sentence, max_new=0)
        f.write(f"Input sentence (Ground Truth):    {sentence}\n")
        f.write(f"Generated:   {output}\n\n")

print(f"Saved generation test to: {filepath}")


Saved generation test to: /content/drive/MyDrive/fMRIwork/generation_test_2025-06-17_04-03-48.txt
